In this notebook, I will:

1. build a __ResNet-15__ architecture from scratch.

2. utilize a learning rate scheduler.

3. eliminate stacking operation from Dataset transforms.

In [1]:
import os
import gc
from random import shuffle

import h5py

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn, optim

from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

from torchvision import transforms as T
from torchvision import models

/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/harkhymadhe/miniforge3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail23torchInternalAssertFailEPKcS2_jS2_RKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# File paths
electron_file = "dataset/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5"
photon_file = "dataset/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5"

In [4]:
# Load data files
electron_data = h5py.File(name = electron_file)
photon_data = h5py.File(name = photon_file)

In [5]:
electron_data.keys()

<KeysViewHDF5 ['X', 'y']>

In [6]:
# Feature shape
electron_data['X'].shape

(249000, 32, 32, 2)

In [7]:
# Target shape
electron_data['y'].shape

(249000,)

In [8]:
# Feature shape
photon_data['X'].shape

(249000, 32, 32, 2)

In [9]:
# Target shape
photon_data['y'].shape

(249000,)

In [10]:
# Target shape
photon_data['y'].shape

(249000,)

In [11]:
def h5_to_numpy(h5_file):
    X, y = h5_file["X"], h5_file["y"]
    return np.array(X), np.array(y)

In [12]:
electron_data = h5_to_numpy(electron_data)
photon_data = h5_to_numpy(photon_data)

In [13]:
data = np.concatenate([electron_data[0], photon_data[0]], axis = 0)
targets = np.concatenate([electron_data[1].reshape(-1, 1), photon_data[1].reshape(-1, 1)], axis = 0)

In [14]:
del electron_data
del photon_data

In [15]:
gc.collect()

0

In [16]:
targets.dtype

dtype('float32')

In [17]:
data[:10].shape

(10, 32, 32, 2)

In [18]:
np.unique(targets)

array([0., 1.], dtype=float32)

In [19]:
targets[:10]

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [20]:
targets[-20:]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)

In [21]:
targets[::2]

array([[1.],
       [1.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
y_train, y_test = train_test_split(pd.DataFrame(targets), test_size = .2, shuffle = True, stratify = targets)

In [24]:
train_indices = y_train.index.values

In [25]:
test_indices = y_test.index.values

In [26]:
train_indices

array([ 49090, 304346, 414673, ..., 241572, 273851, 244299])

In [27]:
test_indices

array([329079,  46482, 120159, ..., 469713, 375212,  94085])

### Data Visualization

In [28]:
data.max()

2.2779698

In [29]:
data.min()

-2.512557

In [30]:
data.mean()

0.00047892798

In [ ]:
data.std()

In [ ]:
data.shape

In [ ]:
np.expand_dims(data[0].mean(axis = -1), axis = -1).shape

In [ ]:
def visualize_particle(index):
    img1 = data[index]
    label1 = targets[index]
    label1 = "electron" if label1 == 1 else "photon"

    img2 = data[-index]
    label2 = targets[-index]
    label2 = "electron" if label2 == 1 else "photon"

    fig, axes = plt.subplots(nrows = 3, ncols = 2, figsize = (10, 7))

    img1 = np.concatenate([img1, np.expand_dims(img1.mean(axis = -1), axis = -1)], axis = -1)
    img2 = np.concatenate([img2, np.expand_dims(img2.mean(axis = -1), axis = -1)], axis = -1)
    
    # Histogram plots
    axes[0,0].hist(img1[:, :, :2].flatten(), bins = 255)
    axes[0,1].hist(img2[:, :, :2].flatten(), bins = 255)

    axes[0,0].set_title(f"Histogram plot of {label1} particle")
    axes[0,1].set_title(f"Histogram plot of {label2} particle")

    axes[0,0].set_xticks([]); axes[0,0].set_yticks([])
    axes[0,1].set_xticks([]); axes[0,1].set_yticks([])

    # Hit energy
    axes[1,0].imshow(img1[:, :, 0])
    axes[1,1].imshow(img2[:, :, 0])

    axes[1,0].set_title(f"Hit energy of {label1} particle")
    axes[1,1].set_title(f"Hit energy of {label2} particle")

    axes[1,0].set_xticks([]); axes[1,0].set_yticks([])
    axes[1,1].set_xticks([]); axes[1,1].set_yticks([])

    # Time to hit

    axes[2,0].imshow(img1[:, :, 1])
    axes[2,1].imshow(img2[:, :, 1])

    axes[2,0].set_title(f"Time to hit for {label1} particle")
    axes[2,1].set_title(f"Time to hit for {label2} particle")
    
    axes[2,0].set_xticks([]); axes[2,0].set_yticks([])
    axes[2,1].set_xticks([]); axes[2,1].set_yticks([])
    
    plt.show(); plt.close("all")

    return

In [ ]:
visualize_particle(index = 10)

In [ ]:
data.shape

data[:, :, :, 0].mean()

data[:, :, :, 0].std()

data[:, :, :, 1].mean()

data[:, :, :, 1].std()

data[:, :, :, 1].shape

(data[:, :, :, 0] + data[:, :, :, 1]).mean()

(data[:, :, :, 0] + data[:, :, :, 1]).std()

In [ ]:
mean = [0.001219672, -0.0002618075, 0.0009578699]

In [ ]:
std = [0.023721104, 0.06738354, 0.07137743]

In [ ]:
mean[0] + mean[1]

In [ ]:
# Set device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data = torch.tensor(data).to(DEVICE)
targets = torch.tensor(targets, dtype = torch.int64).to(DEVICE)

In [ ]:
# Split into train and test samples
train_data, test_data = data[train_indices], data[test_indices]
train_targets, test_targets = targets[train_indices], targets[test_indices]

In [ ]:
del data
del targets

In [ ]:
gc.collect()

In [ ]:
# Generate train and test datasets
train_dataset = TensorDataset(train_data.permute(0, 3, 1, 2), train_targets)
test_dataset = TensorDataset(test_data.permute(0, 3, 1, 2), test_targets)

In [ ]:
test_dataset[0][0].shape

In [ ]:
x = test_dataset[0][0]

In [ ]:
class ParticleDataset(Dataset):
    def __init__(self, dataset, transform = None):
        self.dataset = dataset
        if transform is None:
            transform = T.Compose(
                [
                    T.Lambda(lambda x: torch.cat([x, x.mean(dim = 0, keepdim = True)], dim = 0).squeeze()),
                    T.RandomAdjustSharpness(
                        sharpness_factor = torch.randint(low = 0, high = 10, size = (1,)).item(),
                        p = torch.rand(size = (1,)).item(),
                    ),
                    # T.RandomInvert(),
                    # T.Normalize(mean = mean, std = std)
                ]
            )

        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img, label = self.dataset[index]
        return self.transform(img), label

In [ ]:
# Generate train and test datasets
train_dataset = ParticleDataset(dataset = train_dataset)
test_dataset = ParticleDataset(dataset = test_dataset)

In [ ]:
len(train_dataset)

In [ ]:
len(test_dataset)

In [ ]:
x = test_dataset[1000][0]

In [ ]:
x.shape

In [ ]:
x.max()

In [ ]:
x.min()

Next, I define dataloaders from the datasets.

In [ ]:
BATCH_SIZE = 128

train_dl = DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True)
test_dl = DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle = False)

In [ ]:
s = next(iter(train_dl))

In [ ]:
s[0].shape

In [ ]:
gc.collect()

### Data Modelling

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, in_channel, out_channel, kernel_size = 3, stride = 1, padding = 1, p = .5):
        super().__init__()
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        
        self.kernel_size = kernel_size
        self.padding = padding
        self.stride = stride

        self.p = p
        
        self.conv1 = nn.Conv2d(
            in_channels = self.in_channel,
            out_channels = self.out_channel,
            stride = self.stride,
            kernel_size = self.kernel_size,
            padding = self.padding
        )
        self.bn1 = nn.BatchNorm2d(self.out_channel)
        
        self.relu = nn.ReLU(inplace=True)

        self.dropout = nn.Dropout2d(p = self.p)

        self.conv2 = nn.Conv2d(
            in_channels = self.out_channel,
            out_channels = self.out_channel,
            stride = self.stride,
            kernel_size = self.kernel_size,
            padding = self.padding
        )
        self.bn2 = nn.BatchNorm2d(self.out_channel)

        if self.in_channel != self.out_channel:
            upsample1 = nn.Conv2d(
                in_channels = self.in_channel,
                out_channels = self.out_channel,
                stride = self.stride,
                kernel_size = self.kernel_size,
                padding = self.padding
            )
            upsample2 = nn.Conv2d(
                in_channels = self.out_channel,
                out_channels = self.out_channel,
                stride = self.stride,
                kernel_size = self.kernel_size,
                padding = self.padding
            )

            self.upsample = nn.Sequential(
                upsample1,
                upsample2
            )
        else:
            self.upsample = nn.Identity()
        

    def forward(self, x):
        x_ = self.conv1(x)
        x_ = self.bn1(x_)
        
        x_ = self.relu(x_)
        x_ = self.dropout(x_)

        x_ = self.conv2(x_)
        x_ = self.bn2(x_)
        
        return self.upsample(x) + x_

In [ ]:
class ParticleModel(nn.Module):
    def __init__(self, in_channels, out_channel = 512, out_features = 2):
        super().__init__()
        
        self.in_channels = in_channels
        self.out_channel = out_channel
        self.out_features = out_features
        
        blocks = nn.Sequential()

        for i, channel in enumerate(self.in_channels[:-1]):
            block = BasicBlock(
                in_channel = channel,
                out_channel = self.in_channels[i+1],
                kernel_size = 3,
                stride = 1,
                padding = 1
            )
            blocks.add_module(f"block{i+1}", block)

        self.blocks = blocks

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.LazyLinear(out_features = self.out_channel),
            nn.ReLU(inplace = True),
            nn.LazyLinear(out_features = self.out_features)
        )

    def forward(self, x):
        x = self.blocks(x)
        return self.fc(x)

In [ ]:
image_channels = 3

In [ ]:
in_channels = [image_channels] + [16, 32,] * 3

In [ ]:
in_channels.sort()

In [ ]:
in_channels

In [ ]:
model = ParticleModel(in_channels = in_channels)

In [ ]:
model

In [ ]:
sample_input = torch.randn(8, 3, 32, 32)

In [ ]:
out = model(sample_input)

In [ ]:
out.shape

In [ ]:
for name, p in model.named_parameters():
    print(name)

---

Now, I define a function to initialize model weights.

---

In [ ]:
def initialize_weights(model):
    for (name, weights) in filter(lambda x: x[1].requires_grad, model.named_parameters()):
        # if name.split(".")[-1] not in ["fc", "conv1"]:
        #     continue
        try:
            nn.init.kaiming_normal_(weights, nonlinearity = "relu")
        except:
            nn.init.normal_(weights, mean = 0., std = 0.05)
    
    return model

def get_l2_loss(model):
    return sum([x ** 2 for x in model.parameters()])

In [ ]:
model.to(DEVICE)

model = initialize_weights(model)

In [ ]:
criterion = nn.CrossEntropyLoss().cuda()

In [ ]:
EPOCHS = 20
l2_lambda = .3

criterion = nn.CrossEntropyLoss().to(DEVICE)

# Optimizer hyperparameters
LR = 1e-1
FACTOR = 10
AMSGRAD = False
BETAS = (.9, .999)

In this notebook, the pretrained weights will be finetuned. This is in contrast to the previous one, where the weights were kept frozen. Also, the learing rate is increased from 1e-4 to 1e-3.

In [ ]:
opt = optim.Adam(
    params = [{
        "params" : model.fc.parameters(),
        "lr": LR
    }],
    lr=LR/FACTOR,
    amsgrad = AMSGRAD,
    betas = BETAS,
    weight_decay = l2_lambda
)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(opt, patience = 2, min_lr = 1e-6)

In [ ]:
def get_l2_loss(model):
    l2_loss = torch.tensor(0.).cuda()
    l2_loss += sum(map(lambda x: x.data.pow(2).sum(), filter(lambda x: x.requires_grad, model.parameters())))
    return l2_loss

In [ ]:
def collate_function_dl(batch):

    #xs = batch[0].clone()
    #ys = batch[1].clone()

    xs = [item[0].unsqueeze(0) for item in batch]
    ys = [item[1] for item in batch]
    
    xs = torch.cat(xs, dim=0)

    y = torch.tensor(ys).view(-1, 1)
    
    Xs = [torch.rot90(xs, k = _, dims = [-2, -1]) for _ in range(4)]

    return torch.cat(Xs, dim = 0), torch.cat([y for _ in range(4)], dim = 0).view(-1)

def collate_function(batch):

    xs = batch[0].clone()
    ys = batch[1].clone().view(-1, 1)
    
    Xs = [torch.rot90(xs, k = _, dims = [-2, -1]) for _ in range(4)]

    return torch.cat(Xs, dim = 0), torch.cat([ys for _ in range(4)], dim = 0).view(-1)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
def training_loop(epochs, model, optimizer):
    TRAIN_LOSSES, TEST_LOSSES = [], []
    TRAIN_ACCS, TEST_ACCS = [], []

    for epoch in range(1, epochs + 1):
        train_losses, test_losses = [], []
        train_accs, test_accs = [], []

        model.train() # Set up training mode

        for batch in iter(train_dl):
            # X, y = collate_function(batch)
            X, y = batch
            X, y = X.to(DEVICE), y.view(-1).to(DEVICE)

            with torch.cuda.amp.autocast():
                y_pred = model(X)
            
            # Uncomment the line below if the criterion is nn.NLLLoss()
            # y_pred = torch.log_softmax(y_pred, dim = -1)

            # Compare actual targets and predicted targets to get the loss
            train_loss = criterion(y_pred, y) #+ (l2_lambda * get_l2_loss(model))
            # Backpropagate the loss
            train_loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(train_loss.item())

            train_acc = accuracy_score(y.cpu().numpy(), y_pred.max(dim = -1).indices.cpu().numpy())
            train_accs.append(train_acc)

        scheduler.step(train_loss)
        
        with torch.no_grad(): # Turn off computational graph
            model.eval() # Set model to evaluation mode
            for batch in iter(test_dl):
                # X_, y_ = collate_function(batch)
                X_, y_ = batch
                X_, y_ = X_.to(DEVICE), y_.view(-1).to(DEVICE)
    
                with torch.cuda.amp.autocast():
                    y_pred_ = model(X_)
                
                # Uncomment the line below if the criterion is nn.NLLLoss()
                # y_pred_ = torch.log_softmax(y_pred_, dim = -1)
    
                # Compare actual targets and predicted targets to get the loss
                test_loss = criterion(y_pred_, y_) #+ (l2_lambda * get_l2_loss(model))
                test_losses.append(test_loss.item())

                test_acc = accuracy_score(y_.cpu().numpy(), y_pred_.max(dim = -1).indices.cpu().numpy())
                test_accs.append(test_acc)

        avg_train_loss = sum(train_losses) / len(train_losses)
        avg_test_loss = sum(test_losses) / len(test_losses)

        avg_train_acc = sum(train_accs) / len(train_accs)
        avg_test_acc = sum(test_accs) / len(test_accs)

        print(
            f"Epoch: {epoch} | Train loss: {avg_train_loss: .3f} | Test loss: {avg_test_loss: .3f} |",
            f"Train accuracy: {avg_train_acc: .3f} | Test accuracy: {avg_test_acc: .3f} |"
        )

        TRAIN_LOSSES.append(avg_train_loss)
        TEST_LOSSES.append(avg_test_loss)

        TRAIN_ACCS.append(avg_train_acc)
        TEST_ACCS.append(avg_test_acc)

    # Clear CUDA cache
    torch.cuda.empty_cache()
    torch.clear_autocast_cache()

    return {
        "loss": [TRAIN_LOSSES, TEST_LOSSES],
        "accuracy": [TRAIN_ACCS, TEST_ACCS],
        "model": model
    }

In [ ]:
# Train Resnet-18 with finetuning
model_results = training_loop(epochs = EPOCHS, optimizer = opt, model = model)

In [ ]:
def visualize_results(history, key = None):
    if key is not None:
        TRAIN_RESULTS, TEST_RESULTS = history[key]

        plt.figure(figsize = (10, 3))

        plt.plot(range(EPOCHS), TRAIN_RESULTS, label = f"Training {key.capitalize()}")
        plt.plot(range(EPOCHS), TEST_RESULTS, label = f"Test {key.capitalize()}")

        plt.xlabel("Epochs")
        plt.ylabel(key.capitalize())

        plt.title(key.capitalize() + " Evolution for Train and Test Splits", fontsize = 16)

        plt.legend()
        plt.show(); plt.close("all")
    else:
        TRAIN_LOSSES, TEST_LOSSES = history["loss"]
        TRAIN_ACCS, TEST_ACCS = history["accuracy"]

        fig, ax = plt.subplots(1, 2, figsize = (15, 4))

        ax[0].plot(range(EPOCHS), TRAIN_LOSSES, label = "Training Loss")
        ax[0].plot(range(EPOCHS), TEST_LOSSES, label = "Test Loss")

        ax[0].set_xlabel("Epochs")
        ax[0].set_ylabel("Loss")

        ax[0].set_title("Loss Evolution for Train and Test Splits", fontsize = 16)

        ax[1].plot(range(EPOCHS), TRAIN_ACCS, label = "Training Accuracy")
        ax[1].plot(range(EPOCHS), TEST_ACCS, label = "Test Accuracy")

        ax[1].set_xlabel("Epochs")
        ax[1].set_ylabel("Accuracy")

        ax[1].set_title("Accuracy Evolution for Train and Test Splits", fontsize = 16)

        plt.legend()
        plt.show(); plt.close("all")

    return

In [ ]:
# VGG-13 with finetuning
visualize_results(model_results)